In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from preisach import models, transducer
from instruments import yoko7651
from instruments import zvk
import datetime
import time

reload(zvk)
reload(yoko7651)
reload(models)
reload(transducer)

%cd C:\Users\etudiant.jeip\Documents\tunable-filter\preisach\


# -------------------- PARAMETERS
PATH = r"C:\Users\etudiant.jeip\Documents\tunable-filter\preisach"


C:\Users\etudiant.jeip\Documents\tunable-filter\preisach


In [2]:
def get_max_peaks(amps, f_list):
    if f_list.ndim == 1 : 
        v_shape, _ = amps.shape
        max_freqs = np.zeros(v_shape)
        max_values = np.zeros(v_shape)
        for i in range(v_shape) :
            v_slice = amps[i, :]
            max_freqs[i] = f_list[np.argmax(v_slice)]
            max_values[i] = np.max(v_slice)
        return max_freqs, max_values
    
    elif f_list.ndim == 2:
        v_shape, _ = amps.shape
        max_freqs = np.zeros(v_shape)
        max_values = np.zeros(v_shape)
        for i in range(v_shape) :
            v_slice = amps[i, :]
            max_freqs[i] = f_list[i, np.argmax(v_slice)]
            max_values[i] = np.max(v_slice)
        return max_freqs, max_values
    else : raise ValueError("f_list must have one or two dims.")

def get_band_pass_position(amps_dB, f_list, cutoff = 3):
    if f_list.ndim == 1 : 
        v_shape = amps_dB.shape[0]
        band_freqs = np.zeros((v_shape, 2))
        max_values = np.zeros(v_shape)
        for i in range(v_shape) :
            v_slice = amps_dB[i]
            max_val = np.max(v_slice)
            max_values[i] = max_val
            pass_band = np.nonzero(v_slice >= (max_val - cutoff)) # get indices of values that are less that 3dB from plateau value
            band_freqs[i, :] = f_list[np.min(pass_band)], f_list[np.max(pass_band)]
        return band_freqs, max_values
    
    elif f_list.ndim == 2:
        v_shape, _ = amps_dB.shape
        band_freqs = np.zeros((v_shape, 2))
        max_values = np.zeros(v_shape)
        for i in range(v_shape) :
            v_slice = amps_dB[i, :]
            max_val = np.max(v_slice)
            max_values[i] = max_val
            pass_band = np.nonzero(v_slice >= (max_val - cutoff)) # get indices of values that are less that 3dB from plateau value
            band_freqs[i, :] = f_list[i, np.min(pass_band)], f_list[i, np.max(pass_band)]
        return band_freqs, max_values
    else : raise ValueError("f_list must have one or two dims.")
    
def get_band_pass_single_trace(amps_dB, f_list, cutoff = 3):
    max_val = np.max(amps_dB)
    pass_band = np.nonzero(amps_dB >= (max_val - cutoff)) # get indices of values that are less that 3dB from plateau value
    band_freqs = f_list[np.min(pass_band)], f_list[np.max(pass_band)]
    return band_freqs, max_val

def theoretical_peak_position(V):
    return (38e9/10)*V + 2e9

def lin_V(f):
    return (f - 2e9)*(10/38e9)

def freq_window(f, delta):
    """Gives the freq window to give  to the VNA for input freq f and window width delta (delta in Hz)"""
    if f <= (40e9 - delta/2) :
        return np.array([-delta/2, delta/2]) + f
    else : 
        return np.array([40e9 - delta, 40e9])
    
def ramp_yoko(voltage, delay):
    global CURRENT_YOKO_VOLTAGE
    delta_v = abs(voltage - CURRENT_YOKO_VOLTAGE)
    if delta_v != 0 :
        delta_t = round(delay, 1)
        print(f"GOING TO VOLTAGE {voltage:.1f}, deltaV = {delta_v:.1f} V, delta_t = {delta_t:.1f} s")
        yoko.interval(delta_t)
        yoko.sweep_duration(delta_t)

        with yoko.write_program() as program : 
            program.source_voltage()
            program.range_voltage(12)
            program.voltage(voltage)
        
        yoko.run_program()
        time.sleep(delta_t + 1) # adding one milli second to make sure filter stabilizes

    CURRENT_YOKO_VOLTAGE = voltage
    return None

# Live testing of the precision of the filter in the DC regime
## Setting up brains

In [7]:
%matplotlib widget
########## ----- LOADING DATA

measurement = "preisach_triangle20220727-10h36min28s.npz" # 201² points

#measurement = "preisach_triangle20220726-15h27min49s.npz"

with np.load(measurement, allow_pickle = True) as npzfile:
    zvk_traces = npzfile["zvk_traces"]
    zvk_freqs = npzfile["zvk_freqs"]
    V_values = npzfile["V_values"]
    indices = npzfile["indices"]
    center_freqs = npzfile["center_freqs"]*1e9
    meta = npzfile['meta'].item()

#V_values = np.linspace(0, 10, V_values.shape[0])

V_history = V_values[indices]

########## ----- SETTING UP MODEL

AIM_TOL = 0.5e6
TOTAL_TOL = 12.5e6
preisach_transducer = transducer.Transducer(V_history, center_freqs, tolerance = AIM_TOL)

f_min, f_max = preisach_transducer.output_bounds

########## ----- CONNECTING TO INSTRUMENTS AND SETUP

vna = zvk.Zvk("GPIB0::20::INSTR")
yoko = yoko7651.Yoko7651("GPIB0::16::INSTR")

# setting up VNA
N_FREQ_SWEEP_POINTS = 401 # number of steps for VNA freq sweeps
VNA_WINDOW_WIDTH = 100e6
TRACENAME = "CH1DATA"

vna.sweep_points = N_FREQ_SWEEP_POINTS
vna.power = -15 #dBm
vna.averaging = 0
vna.sweep_count = 1
vna.bandwidth = 10e3
VNA_SWEEP_DUR = 1.1*vna.sweep_duration
vna.single_sweep = True

# ramping Yoko back to 0V 
SMALLEST_STEP_RAMP_TIME = 0.1 # Yoko only allows sweeps to last for 0.1 to 3600 s, with a resolution of 0.1s . 
CURRENT_YOKO_VOLTAGE = 10
ramp_yoko(0, 10)

traces_output = widgets.Output()
history_output = widgets.Output()


def aim_and_plot(f_ghz):
    f = 1e9*f_ghz
    freq_limits = freq_window(f, VNA_WINDOW_WIDTH)
    vna.freq_start_stop = freq_limits
    
    v = preisach_transducer.aim(f)
    predicted_output = preisach_transducer.get_value()

    ramp_yoko(v, 0.1)
    time.sleep(0.02)
    vna.trigger()

    while vna.busy() :
        time.sleep(0.01)

    vna_freqs, vna_z = vna.get_data(trace = TRACENAME)
    plt.close("all")
    plot_band(f_ghz, vna_freqs, vna_z, predicted_output*1e-9)
    plot_history()
    return None

@traces_output.capture(clear_output = True)
def plot_band(f_ghz, freq_list, z_list, preisach_prediction):
    s21_dB = 20*np.log10(np.abs(z_list))
    freq_list_ghz = 1e-9*freq_list
    
    band_freqs_verif, max_values_verif = get_band_pass_single_trace(s21_dB, freq_list_ghz)
    true_center_ghz = np.mean(band_freqs_verif)
    
    y_bounds = [np.min(s21_dB), np.max(s21_dB)]
    fig, ax = plt.subplots(figsize = (8, 8),constrained_layout = True)
    ax.set_title("VNA Trace")
    ax.set_xlabel('f (GHz)')
    ax.set_ylabel(r'Gain (dB)')

    ax.set_xlim(freq_list_ghz[0], freq_list_ghz[-1])
    ax.set_ylim(-60, 0)
    
    ax.axvspan(*band_freqs_verif, color = 'lightgray', alpha = 0.5, label = '3 dB pass band from max')
    ax.plot(freq_list_ghz, s21_dB, color = 'darkorange', label = r"$\left| S_{21}\right|$")

    ax.axvline(f_ghz, color = 'k', ls = '--', label = "Setpoint")
    ax.axvline(preisach_prediction, color = 'tab:red', ls = '--', label = "Predicted position by preisach")
    ax.axvline(true_center_ghz, color = 'darkslateblue', label = "Measured band center")

    ax.legend()
    plt.show(fig)

@history_output.capture(clear_output = True)
def plot_history():
    preisach_history = preisach_transducer.history
    fig2, ax2 = plt.subplots(constrained_layout = True, figsize = (5, 3))
    ax2.set_title("History of the filter")
    ax2.set_xlabel('index (arb)')
    ax2.set_ylabel(r'V (V)')
    ax2.set_ylim(0, 10)
    ax2.grid(True)

    ax2.plot(preisach_history.flatten(), ls = ':', marker = 'o', color = 'tab:red', label = 'Extrema')
    plt.show(fig2)
        
        

Instrument initialized.
VISA resource: GPIB0::20::INSTR
VISA instrument released (GPIB0::20::INSTR).
Instrument initialized.
VISA resource: GPIB0::16::INSTR
VISA instrument released (GPIB0::16::INSTR).
GOING TO VOLTAGE 0.0, deltaV = 10.0 V, delta_t = 10.0 s
Starting to write program...
Finishing program writing.


## Window stuff

In [8]:
h0 = widgets.HBox()
v1 = widgets.VBox()
    

freq_control = widgets.BoundedFloatText(
    value=f_min*1e-9,
    min=f_min*1e-9,
    max=f_max*1e-9,
    step=1e-4,
    description='Setpoint:',
    disabled=False
)

bounds_display = widgets.HTML(
    value=f"Model frequency bounds are {f_min*1e-9:.4f} to {f_max*1e-9:.4f} GHz",
)


filter_update = widgets.interactive_output(aim_and_plot, {'f_ghz': freq_control})

vertical_stack = [bounds_display, freq_control, history_output]
horizontal_stack = [traces_output, v1]

v1.children = vertical_stack
h0.children = horizontal_stack

display(h0)